# Die Webseite der TH Nürnberg Scrapen

### Einleitung

Die Webseite der TH-Nürnberg wirkt als Ausgangspunkt für die Wissensgrundlage des Chatbots.

Conventions: 
- pandas Spaltennamen im Sigular
- Die meißten Links sind keine URLs, da sie lokal sind

In [1]:
import sys
sys.path.append('..')
from bs4 import BeautifulSoup
from bs4.element import Comment
import pandas as pd
from db_init import db_get_df, db_save_df
import glob
import json
import requests
import sys
from tqdm import tqdm

### Scrapen der home Seite

Als Ausganspunkt für unsere Daten nutzen wir die Home Webseite der TH-Nürnberg. (https://www.th-nuernberg.de/)
Diese Website downloaden wir und suchen alle Links auf andere Webseiten und speichern diese Links in eine Liste.
Als nächsten Schritt rufen wir alle Links aus dieser Liste Auf und sammeln wiederum alle Links von jeder dieser Webseiten.
In der daraus resultierenden Liste sortieren wir alle Links aus, die nicht auf die Webseite der TH verweisen.
Dann laden wir alle Dokumente herunter und speichern sie in der Datenbank.

Eine Funktion, die eine URL als parameter nimmt und das HTML file zurückgibt, falls die Seite existiert

In [5]:
def download_html_from_url(url):
    res = requests.get(url)
    html = ""
    if res.ok:
        html =  res.text
    return html

Eine Funktion, die ein HTML file nach links durchsucht und alle gefundenen externen und internen Links zurückgibt.

In [6]:
def get_links_from_one_html(html):
    soup = BeautifulSoup(html,"lxml")
    urls = [a["href"] for a in soup.find_all('a', href=True)]
    return urls

Jetzt geben wir unsere initiale URL an und extrahieren alle Links aus dieser

In [32]:
BASE_URL = "https://www.th-nuernberg.de/"
html = download_html_from_url(BASE_URL)
links = get_links_from_one_html(html)
df = pd.DataFrame({"link": links})
print(*df["link"])

/ # /en/ # /studium-karriere/ /karriere-bei-uns/stellenangebote/ #navcontent9 /einrichtungen-gesamt/in-institute/e-beratung-onlineberatung/ /einrichtungen-gesamt/in-institute/institut-fuer-angewandte-informatik/ /einrichtungen-gesamt/in-institute/institut-fuer-angewandte-wasserstoffforschung-elektro-und-thermochemische-energiesysteme/ /einrichtungen-gesamt/in-institute/institut-fuer-leistungselektronische-systeme-elsys/ /einrichtungen-gesamt/in-institute/institut-fuer-chemie-material-und-produktentwicklung/ /einrichtungen-gesamt/in-institute/institut-fuer-fahrzeugtechnik/ /einrichtungen-gesamt/in-institute/institut-fuer-energie-und-gebaeude-ieg/ /einrichtungen-gesamt/in-institute/institut-fuer-wasserbau-und-wasserwirtschaft/ /einrichtungen-gesamt/in-institute/language-center/ https://ohm-professional-school.de/home/ /einrichtungen-gesamt/in-institute/polymer-optical-fiber-application-center/ /einrichtungen-gesamt/kompetenzzentren/corporate-development-management-accounting-and-financia

### Links filtern

Zunächst können wir alle Links überprüfen, ob sie Parameter mit *?param1=hallo*, oder sections mit *#section* enthalten. Beide Attribute sind für den Download der Webseiten nicht notwendig und 

### Externe Links filtern

Jetzt können wir Mal nachschauen, auf welche externen Seiten die THN Webseite verweist.

In [7]:
def filter_extern_urls(urls):
    filtered_links = []
    for link in urls:
        if link.startswith("http"):
            filtered_links.append(link)

    return filtered_links

In [34]:
extern_links = filter_extern_urls(df["link"])
print("Anzahl externer Links: ", len(extern_links))
print(extern_links)

Anzahl externer Links:  21
['https://ohm-professional-school.de/home/', 'http://www.efi.fh-nuernberg.de/cig', 'http://www.werkstoffanalytik.de', 'http://www.encn.de', 'https://www.studiengangstest.de', 'http://jobboerse.th-nuernberg.de/', 'http://jobboerse.th-nuernberg.de', 'https://www.studiengangstest.de/portal/', 'https://intern.ohmportal.de/index.php?id=25758', 'https://ohm.kh-netzwerk.de/', 'https://jobboerse.th-nuernberg.de', 'https://leonardo-zentrum.de/labs/', 'https://www.hochschuljobboerse.de/studierende/praxistage?mtm_campaign=a-thn-cs&mtm_kwd=ho-s-OhmB%EF%BF%BDhne', 'https://www.th-nuernberg.de/news-archiv/', 'https://de.linkedin.com/school/thnuernberg/', 'https://www.instagram.com/ohm_thnuernberg/', 'https://www.tiktok.com/@ohm_thnuernberg', 'https://www.youtube.com/user/THNuernberg', 'https://twitter.com/TH_Nuernberg', 'https://www.xing.com/companies/technischehochschulenürnberg', 'https://my.ohmportal.de/']


Die Seite verweist also auf alle gängigen Sozial Media Seiten, wie Twitter, youtube, tiktok (der nicht existiert), instagram (der nicht existiert), xing, oft auf die jobbörse mit mehreren Links ins Intranet und auf die Efi fakultät,

### Interne Links filtern

Wir filtern nun noch alle Links heraus, die keine Text-inhalte besitzen oder auf die Selbe Seite referieren.

In [8]:
def filter_intern_links(urls):
    filtered_links = []
    for url in urls:
        if url.startswith("#"):
            continue
        elif url.startswith("http"):
            continue
        elif url.startswith("mailto:"):
            continue
        elif url.startswith("javascript:"):
            continue
        elif url.startswith("&#"): # is encoded mailto
            continue
        elif ".xml" in url:
            continue
        elif ".pdf" in url:
            continue
        elif url == "/":
            continue
        else:
            filtered_links.append(url)
    return filtered_links

In [36]:
intern_links = filter_intern_links(df["link"])
print("Anzahl interner Links: ", len(intern_links))
print(intern_links)

Anzahl interner Links:  296
['/en/', '/studium-karriere/', '/karriere-bei-uns/stellenangebote/', '/einrichtungen-gesamt/in-institute/e-beratung-onlineberatung/', '/einrichtungen-gesamt/in-institute/institut-fuer-angewandte-informatik/', '/einrichtungen-gesamt/in-institute/institut-fuer-angewandte-wasserstoffforschung-elektro-und-thermochemische-energiesysteme/', '/einrichtungen-gesamt/in-institute/institut-fuer-leistungselektronische-systeme-elsys/', '/einrichtungen-gesamt/in-institute/institut-fuer-chemie-material-und-produktentwicklung/', '/einrichtungen-gesamt/in-institute/institut-fuer-fahrzeugtechnik/', '/einrichtungen-gesamt/in-institute/institut-fuer-energie-und-gebaeude-ieg/', '/einrichtungen-gesamt/in-institute/institut-fuer-wasserbau-und-wasserwirtschaft/', '/einrichtungen-gesamt/in-institute/language-center/', '/einrichtungen-gesamt/in-institute/polymer-optical-fiber-application-center/', '/einrichtungen-gesamt/kompetenzzentren/corporate-development-management-accounting-and

### Dublikate entfernen und sortieren

Jetzt können wir die Links filtern, anschließend alphabetisch sortieren und dann Dublikate URLs entfernen.

In [9]:
def sort_and_remove_dublicates(df):
    df = df.sort_values("link")
    df = df.drop_duplicates(subset="link")
    df = df.reset_index(drop=True)
    return df

In [38]:
intern_links = filter_intern_links(df["link"])
df = pd.DataFrame({"link": intern_links})
df = sort_and_remove_dublicates(df)
print("Anzahl interner Links (ohne Dublikate): ", len(intern_links))
print(*df["link"])

Anzahl interner Links (ohne Dublikate):  296
/beratung-services/ /beratung-services/beratungsstellen/ /beratung-services/beratungsstellen/gruendungsberatung/ /beratung-services/beratungsstellen/psychologische-beratung/ /beratung-services/onlineservices/ /beratung-services/serviceeinrichtungen/ /datenschutz/ /einrichtungen-gesamt/abteilungen/digitales-hochschulmanagement/ /einrichtungen-gesamt/abteilungen/finanzabteilung/ /einrichtungen-gesamt/abteilungen/personalabteilung/ /einrichtungen-gesamt/abteilungen/personalentwicklung/ /einrichtungen-gesamt/administration-und-service/akademisches-controlling/ /einrichtungen-gesamt/administration-und-service/bibliothek/ /einrichtungen-gesamt/administration-und-service/hochschulkommunikation-marketing/ /einrichtungen-gesamt/administration-und-service/hochschulkommunikation-marketing/presse-kommunikation/ /einrichtungen-gesamt/administration-und-service/hochschulkommunikation-marketing/presse-kommunikation/ohm-journal/ /einrichtungen-gesamt/admini

### Weitere Seiten nach urls scrapen

Wir habe also 229 Seiten., die wir jetzt alle nochmal nach links durchforsten können.

In [ ]:
for url in tqdm(df["url"]):
    new_urls = get_links_from_one_url("https://www.th-nuernberg.de" + url)
    new_rows = []
    for new_url in new_urls:
        new_rows.append({'url': new_url})
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    

Jetzt können wir wieder schauen, wie viele Interne und externe URLs wir bekommen haben

In [ ]:
print("Anzahl ungefilterter URLs", len(df))
df = sort_and_remove_dublicates(df)
print("Anzahl gefilterter URLs", len(df))
extern_links = filter_extern_urls(df["url"])
print("Anzahl externer URLs", len(extern_links))
intern_links = filter_intern_links(df["url"])
print("Anzahl interner URLs", len(intern_links))

### Links abspeichern

Für unsere weiteren Schritte werden wir immer nur interne Links verwenden, deshalb speichern wir an dieser Stelle mal die internen Links ab.

In [27]:
db_save_df(df, "only_links")

### Downloaden der files

Jetzt können wir mit dem downloaden anfangen.

Diese Funktion Läd nun alle Html files zu den Links herunter und speichert sie im Dataframe neben den "link" in einer Spalte "html".

In [50]:
def download_all_urls(links):
    htmls = []
    for link in tqdm(links):
        url = "https://www.th-nuernberg.de" + link
        html = download_html_from_url(url)
        htmls.append(html)
    df = pd.DataFrame({"link": links, "html": htmls})
    return df

In [51]:
df_new = download_all_urls(df["link"])

100%|██████████| 237/237 [01:25<00:00,  2.77it/s]


Wir können die Daten an dieser Stelle abspeichern.

In [52]:
db_save_df(df_new, "html_iter_01")

### Weitere Iterationsstufen

Wenn wir ab diesem Abschnitt starten können wir die vorher gesammelten Daten neu laden.

In [2]:
df = db_get_df("html_iter_01")

Jetzt können wir die heruntergeladenen HTML files nach weiteren Links durchsuchen und Sie dem Dataframe hinzufügen

In [3]:
def find_all_links_in_html(htmls):
    all_links = []
    for html in tqdm(htmls):
        links = get_links_from_one_html(html)
        [all_links.append(link) for link in links]
    return all_links

In [10]:
all_links = find_all_links_in_html(df["html"])
len(all_links)

100%|██████████| 237/237 [00:13<00:00, 17.61it/s]


71331

Wir haben jetzt also 71331 Links gesammelt, davon sind aber viele Dublikate.

In [11]:
df_new = pd.DataFrame({"link": all_links, "html": None})
df = pd.concat([df, df_new])

In [13]:
df = sort_and_remove_dublicates(df)
len(df["link"])

4043

Gefiltert nach dublikaten haben wir nun also noch 4043 Links

### Iteratives Downloaden

Um den Daten nun weitere Webseiten hinzuzufügen, können wir für jede weitere URL schauen, ob sie schon heruntergeladen wurde. Wenn nicht, dann laden wir sie jetzt herunter.

In [23]:
def update_df_with_html(df):
    new_rows = []
    for index, row in tqdm(df.iterrows()):
        if pd.isna(row['html']) or row['html'] == '':
            # Call the download function with the link from the 'link' column
            url = "https://www.th-nuernberg.de" + row["link"]
            html = download_html_from_url(url)
            new_rows.append({'link': row["link"], 'html': html})

    if new_rows:
        df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    return df

In [24]:
df = update_df_with_html(df)

427it [02:01,  3.51it/s]


ConnectionError: HTTPSConnectionPool(host='www.th-nuernberg.de&', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x125cdbc70>: Failed to resolve 'www.th-nuernberg.de&' ([Errno 8] nodename nor servname provided, or not known)"))

In [ ]:
len(df_ultimate)

In [ ]:
db_save_df(df_ultimate, "html_ultimate")

### Texte extrahieren

Als nächstes müssen wir aus den rohen html dokumenten die unrelevanten Daten aussortieren

In [11]:
df = db_get_df("html_ultimate")

Die nachfolgende Funktion bestimmt, ob ein Beautifulsoup geparsetes html Element sichtbar ist oder nicht.

In [12]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

Jetzt bestimmen wir eine Funktion, die ein HTML als Input bekommt und daraus nur die sichtbaren Zeichen bestimmt.

In [13]:
def get_content(file):
    soup = BeautifulSoup(file,"lxml")

    title = soup.find("title")
    if title:
        title = title.text
    else:
        title = ""

    main = soup.find("main")
    portal = soup.find("div", {'class': 'portal'})
    page_container = soup.find("div", {'class': 'page-wrap'})
    
    visible_texts = ""
    if main:
        # print("main found")
        container = main.find("div" ,{'class': 'container'}, recursive=False)
        if container:
            texts = container.find_all(text=True)
            visible_texts = filter(tag_visible, texts)
        # else:
        #     print("no container") 

    elif portal:
        # print(f"portal found {filename}")
        texts = portal.find_all(text=True)
        visible_texts = filter(tag_visible, texts)
        # print(u" ".join(t.strip() for t in visible_texts))

    elif page_container:
        # print(f"page_container found {filename}")
        container = page_container.find("div" ,{'class': 'container'}, recursive=False)
        # container = page_container.children[2]
        if container:
            texts = container.find_all(text=True)
            visible_texts = filter(tag_visible, texts)


    return {
        "title":    title,
        "text":     u" ".join(t.strip() for t in visible_texts)
    }

In [15]:
parsed_texts = []
titles = []
for html in tqdm(df["html"]):
    content = get_content(html)
    parsed_texts.append(content["text"])
    titles.append(content["title"])

df["text"] = parsed_texts
df["title"] = titles

  0%|          | 0/6599 [00:00<?, ?it/s]/var/folders/sx/swd0dqk97zvbv2327pzfkh800000gn/T/ipykernel_8552/727059889.py:19: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = container.find_all(text=True)
  2%|▏         | 157/6599 [00:08<03:39, 29.28it/s]/var/folders/sx/swd0dqk97zvbv2327pzfkh800000gn/T/ipykernel_8552/727059889.py:26: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = portal.find_all(text=True)
 70%|███████   | 4652/6599 [05:50<03:13, 10.04it/s]  /Users/br/Projects/IT-Ptojekt-Chatbot/daibl/.venv/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you hav

In [20]:

print(df[df["text"] != ""]["url"])

3                                                    None
5                                                    None
6                                                    None
7                                                    None
8                                                    None
                              ...                        
6498    /studium-karriere/zulassung-und-bewerbung/zula...
6499    /studium-karriere/zulassung-und-bewerbung/zula...
6596                /veranstaltungen/calendar/2021/#c7031
6597                /veranstaltungen/calendar/2023/#c7031
6598                /veranstaltungen/calendar/2025/#c7031
Name: url, Length: 2947, dtype: object
